In [1]:
from pathlib import Path
import sys

sys.path.append('..')
from src.processing_utils import *

INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Incompatible GDAL 3.6.2 found on system. Internal GDAL 3.0.0 from distribution will be used.
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.
Currently installed 8.0, available is 9.0.0.
Please visit http://step.esa.int



In [2]:
# Specifying Input Filepaths
work_dir = Path('/data/S1_InSAR_Juukan/')
filepath_1 = Path(work_dir, 'data_raw', 'S1B_IW_SLC__1SDV_20200516T213951_20200516T214019_021612_029081_ACBD.zip')
filepath_2 = Path(work_dir, 'data_raw', 'S1B_IW_SLC__1SDV_20200528T213952_20200528T214019_021787_0295B1_17C1.zip')

In [3]:
# Reading in Files with SNAP
product_1, product_2 = read_products(filepath_1, filepath_2)

INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.


# INSAR Stack Overview

For best results, the Temporal Baseline should be between 6 - 12 days and the Perpendicular Baseline should be between 150 - 300 m.

In [4]:
perpendicular_baseline, temporal_baseline = insar_stack_overview(product_1, product_2)

print(f"Perpendicular Baseline = {perpendicular_baseline:.2f} m")
print(f"Temporal Baseline = {temporal_baseline:.2f} days")

Perpendicular Baseline = -66.80 m
Temporal Baseline = 12.00 days


# TOPSAR Split
Will need to inspect the image products to select the correct bursts for the area of interest.

In [5]:
aoi = [117.02734629478383, -22.493086973432593, 117.31898748693916, -22.649316075632846]
subswath_bursts = get_subswath_burst(filepath_1, filepath_2, aoi)

print(subswath_bursts)

Loaded ZIP file: S1B_IW_SLC__1SDV_20200516T213951_20200516T214019_021612_029081_ACBD.zip
Loaded location grid with 10 bursts and 231 coordinates
Found 6 XML paths
Loaded ZIP file: S1B_IW_SLC__1SDV_20200528T213952_20200528T214019_021787_0295B1_17C1.zip
Found 6 XML paths
{'IW2': {'product_1': (6, 7), 'product_2': (6, 7)}, 'IW1': {'product_1': (6, 8), 'product_2': (6, 8)}}


In [6]:
topsar_split_1 = topsar_split(product_1, 'IW2', 6, 7)
topsar_split_2 = topsar_split(product_2, 'IW2', 6, 7)


100% done.



INFO: org.hsqldb.persist.Logger: dataFileCache open start


100% done.


# Apply Orbit File

In [7]:
orbit_1 = apply_orbit_file(topsar_split_1)
orbit_2 = apply_orbit_file(topsar_split_2)


100% done.

100% done.


# Back Geocoding

In [8]:
back_geocoded = back_geocoding(orbit_1, orbit_2)


100% done.


# Enhanced Spectral Diversity
Only required if more than one burst was selected in the TOPS Split step.

In [9]:
esd = enhanced_spectral_diversity(back_geocoded)

In [10]:
# Suggested to output the 'Phase_ifg_IW2_VV_08Jul2019_02Jul2019' and 'Phase_ifg_IW2_VV_08Jul2019_02Jul2019' bands as RGB images for inspection at this step for verification before proceeding.
rgb_image_path = Path(
                work_dir,
                f"{filepath_1.stem[:25]}_{filepath_2.stem[33:41]}_split_Orb_Stack_esd.png",
            )
write_rgb_image(esd, rgb_image_path)

INFO: org.esa.s1tbx.sentinel1.gpf.SpectralDiversityOp: Shifts written to file: /home/ubuntu/mambaforge/envs/snap-8/snap/.snap/var/log/IW2_range_shifts.json
INFO: org.esa.s1tbx.sentinel1.gpf.SpectralDiversityOp: Estimating azimuth offset for blocks in overlap: 1/1
INFO: org.esa.s1tbx.sentinel1.gpf.SpectralDiversityOp: Shifts written to file: /home/ubuntu/mambaforge/envs/snap-8/snap/.snap/var/log/IW2_azimuth_shifts.json


# Interferogram

If generating Digital Elevation Models, set 'Subtract Topographic Phase' to False.
Otherwise if you are interested in Surface Displacement, set 'Subtract Topographic Phase' to True.

In [11]:
interfero = interferogram(esd, False)


100% done.


# TOPS Deburst

In [12]:
deburst = tops_deburst(interfero)


100% done.


# Goldstein Filtering

In [13]:
filtered = goldstein_phase_filtering(deburst)


100% done.


# Multilooking

In [14]:
multilook = multilooking(filtered)


100% done.


# Saving the Product

In [15]:
pre_snaphu_path = Path(
    work_dir, 'Temp',
    f"{filepath_1.stem[:25]}_{filepath_2.stem[33:41]}_split_Orb_Stack_esd_ifg_deb_flt_ML.dim",
)
write_product(
    product=multilook, filepath=pre_snaphu_path, fileformat="BEAM-DIMAP"
)

Successfully saved: "/data/S1_InSAR_Juukan/Temp/S1B_IW_SLC__1SDV_20200516_20200528_split_Orb_Stack_esd_ifg_deb_flt_ML.dim"
Elapsed time: 37.72 seconds


# SNAPHU Export

Known issue with SNAP 8/9 where SNAPHU Export does not work within SNAPPY.
https://forum.step.esa.int/t/snaphu-export-in-snap-8-rises-indexoutofboundsexception/29278

Required to fall back to SNAP 7 or use SNAP GPT.

Set the StatCostMethod to TOPO for Elevation or DEFO for Displacement.

In [16]:
snaphu_export_dir = Path(work_dir, 'Temp/')

In [17]:
snaphu_exported = snaphu_export_gpt(filepath=pre_snaphu_path, targetfolder=snaphu_export_dir, statcostmode = 'TOPO', initmethod='MST', numprocessors=8)

INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Incompatible GDAL 3.6.2 found on system. Internal GDAL 3.0.0 from distribution will be used.
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.
Currently installed 8.0, available is 9.0.0.
Please visit http://step.esa.int

INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.hsqldb.persist.Logger: dataFileCache open start


Executing operator...
20%....30%....40%....50%....60%....70%....80%....90%.... done.


-- org.jblas INFO Deleting /tmp/jblas1063937971876981473/libquadmath-0.so
-- org.jblas INFO Deleting /tmp/jblas1063937971876981473/libgfortran-4.so
-- org.jblas INFO Deleting /tmp/jblas1063937971876981473/libjblas_arch_flavor.so
-- org.jblas INFO Deleting /tmp/jblas1063937971876981473/libjblas.so
-- org.jblas INFO Deleting /tmp/jblas1063937971876981473


# SNAPHU Unwrapping

In [18]:
snaphu_export_path = Path(snaphu_export_dir, pre_snaphu_path.stem)

In [19]:
unwrapped = snaphu_unwrapping(filepath=snaphu_export_path)


snaphu v2.0.5
27 parameters input from file snaphu.conf (84 lines total)


Logging run-time parameters to file snaphu.log
Creating temporary directory snaphu_tiles_238015
Unwrapping tile at row 0, column 0 (pid 238021)
Unwrapping tile at row 0, column 1 (pid 238024)
Unwrapping tile at row 0, column 2 (pid 238025)
Unwrapping tile at row 0, column 3 (pid 238038)
Unwrapping tile at row 0, column 4 (pid 238044)
Unwrapping tile at row 0, column 5 (pid 238048)
Unwrapping tile at row 0, column 6 (pid 238049)
Unwrapping tile at row 0, column 7 (pid 238050)
Unwrapping tile at row 0, column 8 (pid 238061)
Unwrapping tile at row 0, column 9 (pid 238067)
Unwrapping tile at row 1, column 0 (pid 238071)
Unwrapping tile at row 1, column 1 (pid 238073)
Unwrapping tile at row 1, column 2 (pid 238074)
Unwrapping tile at row 1, column 3 (pid 238085)
Unwrapping tile at row 1, column 4 (pid 238091)
Unwrapping tile at row 1, column 5 (pid 238095)
Unwrapping tile at row 1, column 6 (pid 238096)
Unwrapping tile at row 1, column 7 (pid 238097)
Unwrapping tile at row 1, column 8 (pid 

356 incremental costs clipped to avoid overflow (0.018%)
356 incremental costs clipped to avoid overflow (0.018%)
356 incremental costs clipped to avoid overflow (0.018%)
356 incremental costs clipped to avoid overflow (0.018%)


Integrating secondary flows
Output written to file UnwPhase_ifg_IW2_VV_28May2020_16May2020.snaphu.img
Removing temporary directory snaphu_tiles_238015
SUGGESTION: Try increasing tile overlap and/or size if solution has edge artifacts
Program snaphu done
Elapsed processor time:   0:03:37.63
Elapsed wall clock time:  0:01:46


# SNAPHU Import

In [20]:
snaphu_imported = snaphu_import(multilook, snaphu_export_path)


100% done.


# Phase to Elevation

In [21]:
elevation = phase_to_elevation(snaphu_imported)


100% done.


# Terrain Correction

In [23]:
terrain_corrected = terrain_correction(elevation, "WGS84(DD)", False)


100% done.


# Saving Processed Product

In [24]:
processed_product_path = Path(
    work_dir,
    f"{pre_snaphu_path.stem}_unw_phs_tc.tif",
)

write_product(terrain_corrected, processed_product_path, 'GeoTIFF')

Successfully saved: "/data/S1_InSAR_Juukan/S1B_IW_SLC__1SDV_20200516_20200528_split_Orb_Stack_esd_ifg_deb_flt_ML_unw_phs_tc.tif"
Elapsed time: 16.69 seconds


In [25]:
processed_product_path = Path(
    work_dir,
    f"{pre_snaphu_path.stem}_unw_phs_tc.dim",
)

write_product(terrain_corrected, processed_product_path, 'BEAM-DIMAP')

Successfully saved: "/data/S1_InSAR_Juukan/S1B_IW_SLC__1SDV_20200516_20200528_split_Orb_Stack_esd_ifg_deb_flt_ML_unw_phs_tc.dim"
Elapsed time: 1.04 seconds


In [ ]:
# parameters = HashMap()
# parameters_snaphu = HashMap()
# parameters_snaphu.put(“targetFolder”, str(temp_path))
# inputfile = snappy.ProductIO.readProduct(str(product))
# result_SNE = snappy.GPF.createProduct(“SnaphuExport”, parameters_snaphu, inputfile)
# snappy.ProductIO.writeProduct(result_SNE, temp_path), “Snaphu”)

# infile = temp_path / “snaphu.conf”
# with open(str(infile)) as lines:
# line = lines.readlines()[6]
# snaphu_string = line[1:].strip()
# snaphu_args = snaphu_string.split()
# process = subprocess.Popen(snaphu_args, cwd=str(temp_path))
# process.communicate()
# process.wait()

# unwrapped_list = glob.glob(str(temp_path)) + “/UnwPhase*.hdr”)
# unwrapped_hdr = str(unwrapped_list[0])
# unwrapped_read = snappy.ProductIO.readProduct(unwrapped_hdr)
# snappy.ProductIO.writeProduct(unwrapped_read, str(temp_path / “unwrapped_read.dim”), “BEAM-DIMAP”)
                                                  
# unwrapped = snappy.ProductIO.readProduct(str(temp_path / “unwrapped_read.dim”))
# snaphu_files = snappy.jpy.array(‘org.esa.snap.core.datamodel.Product’, 2)
# snaphu_files[0] = inputfile
# snaphu_files[1] = unwrapped
# result_SI = snappy.GPF.createProduct(“SnaphuImport”, parameters , snaphu_files)
# result_PD = snappy.GPF.createProduct(“PhaseToDisplacement”, parameters , result_SI)
# snappy.ProductIO.writeProduct(result_PD, str(outpath), “BEAM-DIMAP”)